# Imports

In [1]:
import logging

import pandas as pd

import helpers.hdatetime as hdateti
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hpandas as hpandas
import helpers.hprint as hprint
import im_v2.ccxt.data.extract.extractor as imvcdexex
import im_v2.common.data.client.im_raw_data_client as imvcdcimrdc
import im_v2.common.data.qa.dataset_validator as imvcdqdava
import im_v2.common.data.qa.qa_check as imvcdqqach
import im_v2.common.universe as ivcu
import im_v2.common.universe.universe as imvcounun

%load_ext autoreload
%autoreload 2

/app/core/signal_processing/decorrelation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-0c51a42f-d870-4ea0-a2a1-2148b7a42bca.json'
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.5.0' != container_version='1.4.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='CmTask4516_Collect_data_OHLCV_web_socket_data_collection_2'
  hash='f4f98058c'
  # Last commits:
    * f4f98058c Vlad     CmTask4516_Collect_data_OHLCV_web_socket_data_collection (#4548)  (   2 hours ago) Wed Jun 28 14:23:18 2023  (HEAD -> CmTask4516_Collect_data_OHLCV_web_socket_data_collection_2, origin/master, origin/HEAD, master)
    * b773c6aba Toma Jordania CmTask4652: fix broker params sequence (#4653)                

# Read all downloaded websocket data by RawReader

In [3]:
signature = "periodic_daily.airflow.downloaded_200ms.postgres.ohlcv.futures.v7_3.ccxt.binance.v1_0_0"
reader = imvcdcimrdc.RawDataReader(signature, stage="test")
start_timestamp = pd.Timestamp("2023-06-19T17:00:00+00:00")
end_timestamp = pd.Timestamp("2023-06-23T12:00+00:00")
tables_names = [
    "ccxt_ohlcv_futures_3sec",
    "ccxt_ohlcv_futures_7sec",
    "ccxt_ohlcv_futures_10sec",
    "ccxt_ohlcv_futures_15sec"
]

INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3
INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3
INFO  Unable to fetch DB credentials from environment variables: 
	'POSTGRES_HOST'
	Attempting env file method.
INFO  Unable to fetch DB credentials from env file: 
	
################################################################################
* Failed assertion *
'test' in '['local', 'dev', 'prod']'
################################################################################

	Attempting AWS SecretsManager method.
INFO  Fetching secret: test.im_data_db
INFO  Created test DB connection: 
 <connection object at 0x7fa8cd0dd7c0; dsn: 'user=ck_test password=xxx dbname=test.im_data_db host=dev-im-db.cpox8ul7pzan.eu-north-1.rds.amazonaws.com port=5432', closed: 0>
INFO  Enabled connection to the `ccxt_ohlcv_futur

### 3 seconds

In [4]:
reader.table_name = tables_names[0]
binance_ohlcv_data_3sec = reader.load_db_table(start_timestamp, end_timestamp)

INFO  Executing query: 
	SELECT * FROM ccxt_ohlcv_futures_3sec WHERE timestamp >= 1687194000000 AND timestamp <= 1687521600000


/app/helpers/hsql_implementation.py:582: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [5]:
_3sec_timestamp_by_pairs = (
    binance_ohlcv_data_3sec.reset_index()[["currency_pair", "timestamp"]]
    .drop_duplicates()
    .sort_values(by=["currency_pair", "timestamp"]).reset_index(drop=True)
)

### 7 seconds

In [6]:
reader.table_name = tables_names[1]
binance_ohlcv_data_7sec = reader.load_db_table(start_timestamp, end_timestamp)
_7sec_timestamp_by_pairs = (
    binance_ohlcv_data_7sec.reset_index()[["currency_pair", "timestamp"]]
    .drop_duplicates()
    .sort_values(by=["currency_pair", "timestamp"]).reset_index(drop=True)
)

INFO  Executing query: 
	SELECT * FROM ccxt_ohlcv_futures_7sec WHERE timestamp >= 1687194000000 AND timestamp <= 1687521600000


/app/helpers/hsql_implementation.py:582: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


### 10 seconds

In [7]:
reader.table_name = tables_names[2]
binance_ohlcv_data_10sec = reader.load_db_table(start_timestamp, end_timestamp)
_10sec_timestamp_by_pairs = (
    binance_ohlcv_data_10sec.reset_index()[["currency_pair", "timestamp"]]
    .drop_duplicates()
    .sort_values(by=["currency_pair", "timestamp"]).reset_index(drop=True)
)

INFO  Executing query: 
	SELECT * FROM ccxt_ohlcv_futures_10sec WHERE timestamp >= 1687194000000 AND timestamp <= 1687521600000


/app/helpers/hsql_implementation.py:582: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


### 15 seconds

In [8]:
reader.table_name = tables_names[3]
binance_ohlcv_data_15sec = reader.load_db_table(start_timestamp, end_timestamp)
_15sec_timestamp_by_pairs = (
    binance_ohlcv_data_15sec.reset_index()[["currency_pair", "timestamp"]]
    .drop_duplicates()
    .sort_values(by=["currency_pair", "timestamp"]).reset_index(drop=True)
)

INFO  Executing query: 
	SELECT * FROM ccxt_ohlcv_futures_15sec WHERE timestamp >= 1687194000000 AND timestamp <= 1687521600000


/app/helpers/hsql_implementation.py:582: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


# Read bulk downloaded data

In [9]:
signature = "periodic_daily.airflow.downloaded_1min.parquet.ohlcv.futures.v7_3.ccxt.binance.v1_0_0"
reader = imvcdcimrdc.RawDataReader(signature, stage="preprod")
binance_ohlcv_bulk_data = reader.read_data(start_timestamp, end_timestamp)

INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3
INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3
INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3


In [10]:
bulk_timestamp_by_pairs = (
    binance_ohlcv_bulk_data.reset_index(drop=True)[["currency_pair", "timestamp"]]
    .drop_duplicates()
    .sort_values(by=["currency_pair", "timestamp"]).reset_index(drop=True)
)

# Analyze

## Compare 3 seconds vs daily data

Find rows where the value of one the OHLCVs columns does not match between 3 second data and daily data

In [11]:
#Define a funciton to filter mismatched rows
ohlcvs_cols = ["open", "low", "high", "close", "volume"]
def filter_mismatched_rows(row):
    for col in ohlcvs_cols:
        if row[f"{col}_3sec"] != row[f"{col}_daily"]:
            return True
    return False

In [12]:
binance_ohlcv_bulk_data = binance_ohlcv_bulk_data.reset_index(drop=True)
binance_ohlcv_data_3sec = binance_ohlcv_data_3sec.reset_index(drop=True)

In [13]:
_3sec_daily = pd.merge(
    binance_ohlcv_bulk_data,
    binance_ohlcv_data_3sec,
    on=["currency_pair","timestamp"],
    how="inner",
    suffixes=["_daily", "_3sec"]
)

In [14]:
_3sec_daily_mismatch = _3sec_daily[_3sec_daily.apply(filter_mismatched_rows, axis=1)]
_3sec_daily_mismatch.shape

(0, 20)

As we can see there is not mismatch.

### Date ranges

In [ ]:
_3sec_timestamp_by_pairs.timestamp.agg(["min", "max"]).apply(pd.to_datetime, unit="ms")

In [ ]:
_7sec_timestamp_by_pairs.timestamp.agg(["min", "max"]).apply(pd.to_datetime, unit="ms")

In [ ]:
_10sec_timestamp_by_pairs.timestamp.agg(["min", "max"]).apply(pd.to_datetime, unit="ms")

In [ ]:
_15sec_timestamp_by_pairs.timestamp.agg(["min", "max"]).apply(pd.to_datetime, unit="ms")

In [ ]:
bulk_timestamp_by_pairs.timestamp.agg(["min", "max"]).apply(pd.to_datetime, unit="ms")

In [ ]:
max(bulk_timestamp_by_pairs.timestamp)

In [ ]:
_3sec_timestamp_by_pairs = _3sec_timestamp_by_pairs[_3sec_timestamp_by_pairs.timestamp <= 1687478340000]

In [ ]:
_7sec_timestamp_by_pairs = _7sec_timestamp_by_pairs[_7sec_timestamp_by_pairs.timestamp <= 1687478340000]

In [ ]:
_10sec_timestamp_by_pairs = _10sec_timestamp_by_pairs[_10sec_timestamp_by_pairs.timestamp <= 1687478340000]

In [ ]:
_15sec_timestamp_by_pairs = _15sec_timestamp_by_pairs[_15sec_timestamp_by_pairs.timestamp <= 1687478340000]

### 3 sec

In [ ]:
len(_3sec_timestamp_by_pairs)

In [ ]:
len(bulk_timestamp_by_pairs)

In [ ]:
_3sec_match = pd.merge(
    bulk_timestamp_by_pairs, _3sec_timestamp_by_pairs,
    how="inner",
    on=["currency_pair", "timestamp"]
)

In [ ]:
_3sec_match.shape

In [ ]:
_3sec_not_match = pd.merge(
    bulk_timestamp_by_pairs,
    _3sec_timestamp_by_pairs,
    on=['currency_pair','timestamp'],
    how="outer",
    indicator=True
)
_3sec_not_match = _3sec_not_match[_3sec_not_match['_merge']=='left_only']

In [ ]:
_3sec_not_match.shape

### 7 sec

In [ ]:
len(_7sec_timestamp_by_pairs)

In [ ]:
len(bulk_timestamp_by_pairs)

In [ ]:
pd.merge(
    bulk_timestamp_by_pairs, _7sec_timestamp_by_pairs,
    how="inner",
    on=["currency_pair", "timestamp"]
).shape

In [ ]:
_7sec_not_match = pd.merge(
    bulk_timestamp_by_pairs,
    _7sec_timestamp_by_pairs,
    on=['currency_pair','timestamp'],
    how="outer",
    indicator=True
)
_7sec_not_match = _7sec_not_match[_7sec_not_match['_merge']=='left_only']

In [ ]:
_7sec_not_match.shape

### 10 sec

In [ ]:
len(_10sec_timestamp_by_pairs)

In [ ]:
len(bulk_timestamp_by_pairs)

In [ ]:
pd.merge(
    bulk_timestamp_by_pairs, _10sec_timestamp_by_pairs,
    how="inner",
    on=["currency_pair", "timestamp"]
).shape

In [ ]:
_10sec_not_match = pd.merge(
    bulk_timestamp_by_pairs,
    _10sec_timestamp_by_pairs,
    on=['currency_pair','timestamp'],
    how="outer",
    indicator=True
)
_10sec_not_match = _3sec_not_match[_10sec_not_match['_merge']=='left_only']

In [ ]:
_10sec_not_match.shape

### 15 sec

In [ ]:
len(_15sec_timestamp_by_pairs)

In [ ]:
len(bulk_timestamp_by_pairs)

In [ ]:
pd.merge(
    bulk_timestamp_by_pairs, _15sec_timestamp_by_pairs,
    how="inner",
    on=["currency_pair", "timestamp"]
).shape

In [ ]:
_15sec_not_match = pd.merge(
    bulk_timestamp_by_pairs,
    _15sec_timestamp_by_pairs,
    on=['currency_pair','timestamp'],
    how="outer",
    indicator=True
)
_15sec_not_match = _15sec_not_match[_15sec_not_match['_merge']=='left_only']

In [ ]:
_15sec_not_match.shape